In [8]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd

from pa_lib.file import (
    project_dir,
    load_bin,
    load_xlsx,
    store_bin,
)
from pa_lib.data import as_dtype, dtFactor
from pa_lib.util import cap_words, collect

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data sets

In [2]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_bin("spr_data.feather")

16:49:34 [INFO] Started loading binary file ...
16:49:34 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
16:49:35 [INFO] ... finished loading binary file in 0.3s (1.02s CPU)
16:49:35 [INFO] Started loading binary file ...
16:49:35 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.feather
16:49:35 [INFO] ... finished loading binary file in 0.01s (0.0s CPU)


# Filter data for merging

In [3]:
spr_data.dropna(subset=["PF"], inplace=True)

# Find mapping of Flächen to Stations

In [11]:
pd.DataFrame(dict(Station=ax_data.Station.unique()))

,Station
0,Aarau
1,Basel SBB
2,Bellinzona
3,Bern
4,Biel/Bienne
5,Brig
6,Chur
7,Fribourg
8,Genève Aéroport
9,Genève Cornavin


In [5]:
spr_data.columns

Index(['index', 'WT', 'Flaeche_ID', 'Anbieter', 'Spr_Flaeche_ID', 'Time',
       'Count', 'Nummer', 'PF', 'Position-Nr.', 'Gebiet', 'Gebietsbez.',
       'Gebiet Code', 'Adresszusatz', 'STM', 'EN-Nr.', 'NG-Nr.'],
      dtype='object')

## Problem cases: ZH, GE, VD (> 1 station each)

In [6]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "ZH", "Adresszusatz"], sep=" | ")

'Gleis 2, Sektor C, Fahrtreppenaufgang | Gleis 3, Sektor C, Fahrtreppenaufgang | Gleis 31/32, Sektor B, Aufgang Halle Löwenstrasse | Gleis 31/32, Sektor B, Wartezone | Gleis 33/34, Sektor B, Aufgang Halle Löwenstrasse | Gleis 33/34, Sektor B, Wartezone | Gleis 4, Sektor A, vis-à-vis Hausperron | Gleis 4, Sektor B, vis-à-vis Hausperron | Gleis 4, Sektor C, vis-à-vis Hausperron | Gleis 41, Linien 5,6,7,9,12, Sektor B | Gleis 41, Linien 5,6,7,9,12, Sektor C | Gleis 44, Linien 5,6,7,9,12, Sektor B | Gleis 44, Linien 5,6,7,9,12, Sektor C | Halle Kopfperron | Halle Landesmuseum, Abgänge S-Bahn Perron Gleis 41+42 / 43+44 | Halle Löwenstrasse, Aufgang Gleis 6 | Halle Löwenstrasse, Aufgang Gleis 7 | Halle Löwenstrasse, Aufgang Shopville | Halle Museumstrasse, Passage Bahnhofstrasse | Halle Museumstrasse, Passage Löwenstrasse | Haupzugang, Kiosk | Passage Löwenstrasse | Passage Sihlquai, Liftschacht Gleis 10 | Passage Sihlquai, Liftschacht Gleis 11 | Passage Sihlquai, Liftschacht Gleis 6 | Passa

In [15]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "ZH", "Position-Nr."], sep=" | ")

'1DA01 | 1DC01 | 1DE01 | 1DH01 | 1DH02 | 8DO01 | 8DP01 | 8DQ01 | 8DR01 | DA01 | DB01 | DC01 | DD01 | DE01 | DE03 | DE11 | DF01 | DF03 | DF11 | DG01 | DG03 | DH01 | DL01 | DM01 | DN01 | DO01'

In [10]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "GE", "Adresszusatz"], sep=" | ")

'Accès hall central, accès Passage Aile Ouest, shopping | Accés Passage Aile Est, hall de la gare, shopping | Entrée/sortie hall de la gare, automate à billets | Hall de la gare, accès Passage Aile Est , shopping, automate à billets, (AG) | Passage Aile Est, voies 3 - 4 , shopping | Passage Aile Est, voies 6 - 7, shopping | Passage Aile Ouest, accès Metroshopping, shopping | Passage Aile Ouest, voies 1 - 2, kiosque, shopping'

In [19]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "GE", "Spr_Flaeche_ID"], sep=" | ")

'1680802 | 1680803 | 1680804 | 1680805 | 1680806 | 1680807 | 1680903 | 1680904 | 1690726'

In [12]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "VD", "Adresszusatz"], sep=" | ")

'Accès passage ouest, shop | Entrée est, accès passage est, kiosque, shop | Hall principal, accès quai, voie 1, kiosque, shop | Passage entrée est - hall principal, shop | Place de la Gare, entrée Quai direction Ouchy | Quai direction Croisettes, porte palière | Quai direction Croisettes, retour nord | Quai direction Ouchy, porte palière | Sortie est, kiosque, shop'